Hypothesis: Participants who have an average of 3 locations per week will have higher stress algorithms than people who use an average of 4 or more locations per week. 

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt  # To visualize
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
SIX_MONTH_DAILY_AM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily AM /7103000 (3).csv"
SIX_MONTH_DAILY_PM_PATH_1 =  "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily PM /7106000 (2).csv"
SIX_MONTH_DAILY_PM_PATH_2 = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily PM /7592000.csv"
SIX_MONTH_DAILY_END_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily END/7591000.csv"
SIX_MONTH_DAILY_END_PATH2 = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Daily END/7108000 (2).csv"
SIX_MONTH_FRIDAY_AM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday AM /7105000 (2).csv"
SIX_MONTH_FRIDAY_PM_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday PM/7107000 (2).csv"
SIX_MONTH_FRIDAY_END_PATH = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Friday END/7109000 (2).csv"


In [ ]:
SIX_MONTH_DAILY_AM = pd.read_csv(SIX_MONTH_DAILY_AM_PATH)
SIX_MONTH_DAILY_PM = pd.concat([pd.read_csv(SIX_MONTH_DAILY_PM_PATH_1, encoding='cp1252'), pd.read_csv(SIX_MONTH_DAILY_PM_PATH_2)])
SIX_MONTH_DAILY_END = pd.concat([pd.read_csv(SIX_MONTH_DAILY_END_PATH), pd.read_csv(SIX_MONTH_DAILY_END_PATH2)])
SIX_MONTH_FRIDAY_AM = pd.read_csv(SIX_MONTH_FRIDAY_AM_PATH)
SIX_MONTH_FRIDAY_PM = pd.read_csv(SIX_MONTH_FRIDAY_PM_PATH, encoding='cp1252')
SIX_MONTH_FRIDAY_END = pd.read_csv(SIX_MONTH_FRIDAY_END_PATH)

In [ ]:
import datetime as dt
def get_week():
  begin = dt.datetime(2022, 4, 24)
  end = dt.datetime.now()
  week_dict = {}
  week_num = 0
  while begin < end:
    curBeg = begin
    curEnd = begin + dt.timedelta(days = 7)
    begin = curEnd
    week_dict[curEnd] = week_num
    week_num += 1
  return week_dict

week_dict = get_week()

In [ ]:
def split_date_time(DATA):
  DATA[["date", "exact_time"]] = DATA["local_time"].str.split(" ", expand=True)
  DATA = DATA.drop(columns = ['local_time'])
  DATA[["Year", "Month", "Day"]] = DATA["date"].str.split("-", expand=True)
  DATA = DATA.drop(columns = ["date"])
  return DATA

SIX_MONTH_DAILY_AM = split_date_time(SIX_MONTH_DAILY_AM)
SIX_MONTH_DAILY_PM = split_date_time(SIX_MONTH_DAILY_PM)
SIX_MONTH_DAILY_END = split_date_time(SIX_MONTH_DAILY_END)
SIX_MONTH_FRIDAY_AM = split_date_time(SIX_MONTH_FRIDAY_AM)
SIX_MONTH_FRIDAY_PM = split_date_time(SIX_MONTH_FRIDAY_PM)
SIX_MONTH_FRIDAY_END = split_date_time(SIX_MONTH_FRIDAY_END)

In [ ]:
SIX_MONTH_DAILY_MERGE_1 = pd.merge(SIX_MONTH_DAILY_AM, SIX_MONTH_DAILY_PM, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_DAILY = pd.merge(SIX_MONTH_DAILY_MERGE_1, SIX_MONTH_DAILY_END, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_FRIDAY_MERGE_1 = pd.merge(SIX_MONTH_FRIDAY_AM, SIX_MONTH_FRIDAY_PM, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])
SIX_MONTH_FRIDAY = pd.merge(SIX_MONTH_FRIDAY_MERGE_1, SIX_MONTH_FRIDAY_END, how = "inner", on =  ["mbl_cod", "Year", "Month", "Day"])

In [ ]:
SIX_MONTH_DAILY_LOCATION = SIX_MONTH_DAILY[['mbl_cod', 'LOCATION_AM', 'LOCATION_PM', 'LOCATION_END', 'Year', 'Month', 'Day']]
SIX_MONTH_FRIDAY_LOCATION = SIX_MONTH_FRIDAY[['mbl_cod', 'LOCATION_AM', 'LOCATION_PM', 'LOCATION_END', 'Year', 'Month', 'Day']]
SIX_MONTH_DAILY_LOCATION['Week Number'] = None
SIX_MONTH_FRIDAY_LOCATION['Week Number'] = None

In [ ]:
def week_num(row):
  datetime = dt.datetime(int(row[4]), int(row[5]), int(row[6]))
  day_of_week = datetime.weekday()
  offset = 6 - day_of_week
  new_date = datetime + dt.timedelta(days = offset)
  row[-1] = week_dict[new_date]
  return row


SIX_MONTH_DAILY_LOCATION = SIX_MONTH_DAILY_LOCATION.apply(week_num, axis = 1)
SIX_MONTH_FRIDAY_LOCATION = SIX_MONTH_FRIDAY_LOCATION.apply(week_num, axis = 1)

In [ ]:
SIX_MONTH_LOCATION = pd.merge(SIX_MONTH_DAILY_LOCATION, SIX_MONTH_FRIDAY_LOCATION, on = ['mbl_cod', 'Week Number'], how = "left")

In [ ]:
from collections import Counter
grouped = SIX_MONTH_LOCATION.groupby(['mbl_cod', 'Week Number'])

new_df = pd.DataFrame()
mbl_cod = []
week_number = []
distinct = []
for key, item in grouped:
    a = grouped.get_group(key)
    a = a.dropna()
    if a.shape[0] == 0:
      continue
    mbl_cod.append(list(a['mbl_cod'])[0])
    am_data = list(a['LOCATION_AM_x'])
    pm_data = list(a['LOCATION_PM_x'])
    end_data = list(a['LOCATION_END_x'])
    friday_am_data = list(a['LOCATION_AM_y'])
    friday_pm_data = list(a['LOCATION_PM_y'])
    friday_end_data = list(a['LOCATION_END_y'])
    week_number.append(list(a['Week Number'])[0])
    all_data = am_data + pm_data + end_data + friday_am_data + friday_pm_data + friday_end_data
    all_data = [int(x) for x in all_data if isinstance(x, float) or isinstance(x, int)]
    distinct_data = len(Counter(all_data).keys())
    distinct.append(distinct_data)

In [ ]:
new_df['mbl_cod'] = mbl_cod
new_df['Week Number'] = week_number
new_df['Number of Locations'] = distinct
new_df.head()

,mbl_cod,Week Number,Number of Locations
0,11822993,2,5
1,11822993,10,3
2,11822993,12,4
3,11822993,13,3
4,17180706,4,3


In [ ]:
import os
dir_path_garmin = "/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/DATA - Working From the Home Environment /6 Month Data/Garmin Data"
print(os.listdir(dir_path_garmin))
all_data = None
for file in os.listdir(dir_path_garmin):
  mbr_col = file.split(".")[1].split(" ")[0]
  full_path = os.path.join(dir_path_garmin, file)
  df = pd.read_csv(full_path)
  df['mbr_col'] = mbr_col
  df = df[df['data_type'] == "stress"]
  df = df[df['val'] != 0]
  if isinstance(all_data, pd.core.frame.DataFrame):
    all_data = pd.concat([all_data, df])
  else:
    all_data = df
    


['garmin.50667687 3.csv', 'garmin.82878753 3.csv', 'garmin.81914178 3.csv', 'garmin.58500979 3.csv', 'garmin.47685985 3.csv', 'garmin.81875100 3.csv', 'garmin.70398973 3.csv', 'garmin.55448394 2.csv', 'garmin.68415107 3.csv', 'garmin.26141560 2.csv', 'garmin.86570707 3.csv', 'garmin.27361835 2.csv', 'garmin.71681441 3.csv', 'garmin.82420964 3.csv', 'garmin.11822993 3.csv', 'garmin.54042771 4.csv', 'garmin.38656882 3.csv', 'garmin.20763027 2.csv', 'garmin.17180706 2.csv', 'garmin.91556555 3.csv', 'garmin.17309235 2.csv', 'garmin.77253909 3.csv', 'garmin.55508636 4.csv', 'garmin.49669568 3.csv', 'garmin.62547526 3.csv', 'garmin.80515680 3.csv', 'garmin.81862952 3.csv', 'garmin.38876664 3.csv', 'garmin.66958688 3.csv', 'garmin.23916703 2.csv', 'garmin.71552354 3.csv', 'garmin.38886354 3.csv', 'garmin.22541511 2.csv', 'garmin.33075391 2.csv', 'garmin.37720972 3.csv', 'garmin.20126808 2.csv', 'garmin.64811087 3.csv', 'garmin.58947714 3.csv', 'garmin.70975009 3.csv', 'garmin.27148444 2.csv',

In [ ]:
all_data.to_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/garmin_combined.csv")

In [ ]:
all_data_dropped = all_data[['mbr_col', 'dte_tme', 'val']]

In [ ]:
all_data_dropped['year'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).year
all_data_dropped['month'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).month
all_data_dropped['day'] = pd.DatetimeIndex(all_data_dropped['dte_tme']).day
all_data_dropped = all_data_dropped.drop(columns = ['dte_tme'])

In [ ]:
def week_num(row):
  datetime = dt.datetime(int(row[2]), int(row[3]), int(row[4]))
  day_of_week = datetime.weekday()
  offset = 6 - day_of_week
  new_date = datetime + dt.timedelta(days = offset)
  row[-1] = week_dict[new_date]
  return row


all_data_dropped['Week Number'] = None
all_data_dropped = all_data_dropped.apply(week_num, axis = 1)

In [ ]:
grouped_data = pd.read_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/grouped_garmin.csv")
grouped_data = grouped_data.rename(columns = {"mbr_col" : "mbl_cod", "val" : "min_number", "val.1" : "max_number", "val.2": "mean_number"})

In [ ]:
final_df = pd.merge(new_df, grouped_data, on = ['mbl_cod', 'Week Number'])
final_df_analysis = final_df[['Number of Locations', 'min_number', 'max_number', 'mean_number']]

In [ ]:
final_df_analysis['GreaterFour'] = None

def greater_four(row):
  row[-1] = 0 if row[0] < 4 else 1
  return row

final = final_df_analysis.apply(greater_four, axis = 1)

In [ ]:
final.to_csv("/content/gdrive/Shareddrives/CS 506 Project (Work from Home)/Data/full_df.csv")

In [ ]:
get_vals = list(final['mean_number'])
get_vals = [str(x) for x in get_vals]
get_vals = [float(x) for x in get_vals]
final['mean_number'] = get_vals

get_vals = list(final['max_number'])
get_vals = [str(x) for x in get_vals]
get_vals = [float(x) for x in get_vals]
final['max_number'] = get_vals



In [ ]:
grouped = final.groupby('GreaterFour').mean_number.mean()

In [ ]:
import seaborn as sb  
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot as plt
from scipy.stats import norm, uniform
import numpy

In [ ]:
import plotly.express as px

fig = px.box(final, x="GreaterFour", y="mean_number")
fig.show()

In [ ]:
df1 = final[final['GreaterFour'] == 0]
df2 = final[final['GreaterFour'] == 1]

In [ ]:
list_fourmore = list(df1['mean_number'])
list_threeless = list(df2['mean_number'])

In [ ]:
from scipy.stats import f_oneway
stat, p = f_oneway(list_fourmore, list_threeless)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('We can not reject the null hypothesis')
else:
	print('We reject the hypothesis')

stat=0.269, p=0.604
We can not reject the null hypothesis


In [ ]:
from scipy.stats import ttest_rel
from scipy import stats

stats.ttest_ind(list_fourmore, list_threeless, equal_var = False)

Ttest_indResult(statistic=0.5779741361510294, pvalue=0.5638051647914657)

In [ ]:
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
import pandas as pd

def welch_ttest(x1, x2,alternative):
    
    n1 = len(x1)
    n2 = len(x2)
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    
    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)
    
    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2
    
    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))
    
    # two side t-test
    p = 2 * t.cdf(-abs(tstat), df)
    
    # upper and lower bounds
    lb = delta - t.ppf(0.975,df)*pooled_se 
    ub = delta + t.ppf(0.975,df)*pooled_se
  
    return pd.DataFrame(np.array([tstat,df,p,delta,lb,ub]).reshape(1,-1),
                         columns=['T statistic','df','pvalue 2 sided','Difference in mean','lb','ub'])

In [ ]:
print(welch_ttest(list_threeless,list_fourmore,"equal"))

   T statistic          df  pvalue 2 sided  Difference in mean        lb  \
0    -0.577974  248.395881        0.563805           -0.691253 -3.046833   

         ub  
0  1.664327  
